<a href="https://colab.research.google.com/github/jayantc14/Yelp_Data_Analysis/blob/main/BERT_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
!pip install tokenization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 29.7 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
     |████████████████████████████████| 61 kB 482 kB/s 
     |████████████████████████████████| 895 kB 44.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
% pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [ ]:
import tensorflow_hub as hub
import tokenization
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/yelp_dataset/yelp_3cities_refined2.csv")


In [ ]:
cols = ['business_id', 'city', 'state', 'user_id', 'useful_x',
       'date', 'useful_y', 'fans']

In [ ]:
df.drop(cols,axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#y = df['review_stars']
#X = df.drop("DEATH_EVENT", axis = 1)
X = df.drop(["review_stars"], axis = 1)
train, test = train_test_split(df, test_size=0.2, random_state=25)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(5, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
import numpy as np

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
import keras

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
!pip install keras.utils

  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=c7ebd7752a28c1a21717c7f3e90ad72335fa9b3c3624081a7a50790776df18d6
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils


In [ ]:
#from keras.utils import to_categorical

In [ ]:
train

,review_stars,text
63082,5,This is one of my favorite places in Boston. I...
139977,5,This place is so underrated. The pizzas are d...
67546,5,Highly recommend visiting this lovely brewery!...
146484,1,Extremely underwhelmed with this experience. W...
1769,4,Gayle and I stayed here the night before board...
...,...,...
130365,5,This place is incredibly busy but after eating...
92303,1,Third time and I give up. I have visited this ...
6618,5,La Merced is really great. The chile relleno b...
90430,2,Bar service was great.. Bouncer made unnecessa...


In [ ]:
max_len = 150
train_input = bert_encode(train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.text.values, tokenizer, max_len=max_len)


In [ ]:
train.reset_index(drop=True,inplace = True)

In [ ]:
for i in range(len(train)):
    if train.loc[i,'review_stars'] == 1:
        train.loc[i,'review_stars'] = 0
    if train.loc[i,'review_stars'] == 2:
        train.loc[i,'review_stars'] = 1
    if train.loc[i,'review_stars'] == 3:
        train.loc[i,'review_stars'] = 2
    if train.loc[i,'review_stars'] == 4:
        train.loc[i,'review_stars'] = 3
    if train.loc[i,'review_stars'] == 5:
        train.loc[i,'review_stars'] = 4

In [ ]:
train

,review_stars,text
0,4,This is one of my favorite places in Boston. I...
1,4,This place is so underrated. The pizzas are d...
2,4,Highly recommend visiting this lovely brewery!...
3,0,Extremely underwhelmed with this experience. W...
4,3,Gayle and I stayed here the night before board...
...,...,...
158354,4,This place is incredibly busy but after eating...
158355,0,Third time and I give up. I have visited this ...
158356,4,La Merced is really great. The chile relleno b...
158357,1,Bar service was great.. Bouncer made unnecessa...


In [ ]:
train_labels = tf.keras.utils.to_categorical(train.review_stars.values, num_classes=None)

In [ ]:
train_labels

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [ ]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 150)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 150)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 150)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 150, 768)]                'input_mask[0][0]',         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    train_input, train_labels, 
    validation_split=0.2,
    epochs=1,
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1)

3959/3959 [==============================] - ETA: 0s - loss: 0.7507 - accuracy: 0.6996
Epoch 00001: val_accuracy improved from -inf to 0.74700, saving model to model.h5
3959/3959 [==============================] - 8206s 2s/step - loss: 0.7507 - accuracy: 0.6996 - val_loss: 0.6172 - val_accuracy: 0.7470


In [ ]:
model.load_weights('model.h5')
y_pred = model.predict(test_input)

In [ ]:
y_pred

array([[5.8790433e-01, 3.1491438e-01, 5.3351179e-02, 2.0844370e-02,
        2.2985743e-02],
       [2.4690153e-03, 1.5460904e-03, 4.4131973e-03, 1.8865427e-01,
        8.0291742e-01],
       [8.7869307e-04, 3.9983299e-04, 1.4273619e-03, 6.4487122e-02,
        9.3280697e-01],
       ...,
       [1.9002199e-04, 6.6403336e-05, 1.8864882e-04, 9.0245418e-03,
        9.9053043e-01],
       [7.6646799e-01, 2.1132703e-01, 1.1656546e-02, 5.9031276e-03,
        4.6453644e-03],
       [9.7816497e-01, 1.7594555e-02, 9.1842387e-04, 1.9372458e-03,
        1.3847969e-03]], dtype=float32)

In [ ]:
test

,review_stars,text
23146,3,Well this was one of our favorite seafood rest...
27022,4,This Laughing Planet location is definitely my...
163469,5,"Soooo good. I've been twice in one week, defin..."
11507,4,We came here because the Pho French Dip looked...
67940,5,What an incredible meal. Honestly the best I'v...
...,...,...
38537,1,Mystery meat! Awful experience here. Meat in t...
6624,3,My partner and I went here for a small celebra...
144662,5,I love this place! Great food and beautiful v...
55348,1,Went yesterday... gross. We couldn't eat our m...


In [ ]:
y_test = y_test.to_numpy()

In [ ]:
y_test

array([3, 4, 5, ..., 5, 1, 1])

In [ ]:
y_hat = y_pred.argmax(axis = 1)[:,None]

In [ ]:
y_hat

array([[0],
       [4],
       [4],
       ...,
       [4],
       [0],
       [0]])

In [ ]:
y_hat2 = y_hat.flatten()

In [ ]:
y_hat2

array([0, 4, 4, ..., 4, 0, 0])

In [ ]:
test.reset_index(drop=True,inplace = True)

In [ ]:
for i in range(len(test)):
    if test.loc[i,'review_stars'] == 1:
        test.loc[i,'review_stars'] = 0
    if test.loc[i,'review_stars'] == 2:
        test.loc[i,'review_stars'] = 1
    if test.loc[i,'review_stars'] == 3:
        test.loc[i,'review_stars'] = 2
    if test.loc[i,'review_stars'] == 4:
        test.loc[i,'review_stars'] = 3
    if test.loc[i,'review_stars'] == 5:
        test.loc[i,'review_stars'] = 4

In [ ]:
y_test = test['review_stars']

In [ ]:
y_test = y_test.to_numpy()

In [ ]:
y_test

array([2, 3, 4, ..., 4, 0, 0])

In [ ]:
accuracy_score(y_test, y_hat2)

0.7463753473099267

In [ ]:
from sklearn import metrics

In [ ]:
print(metrics.classification_report(y_test, y_hat2))

              precision    recall  f1-score   support

           0       0.70      0.87      0.78      4473
           1       0.45      0.42      0.43      2782
           2       0.56      0.45      0.50      3715
           3       0.60      0.46      0.52      7422
           4       0.85      0.92      0.88     21198

    accuracy                           0.75     39590
   macro avg       0.63      0.62      0.62     39590
weighted avg       0.73      0.75      0.73     39590

